**Урок 5**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4901bbc705e6556f3bf89406d09d7d146c495a55cf1f0e1d6fd581258cd68c3f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master('local[4]').appName('joins_hw').getOrCreate()

**Joins**

In [3]:
# Создаём датасет для примеров
dataset_1 = [
  {
    'key_1' : 'abc',
    'value_1' : 10,
    'value_2' : 20
  },
  {
    'key_1' : 'def',
    'value_1' : 100,
    'value_2' : 300
  }
]

dataset_2 = [
  {
    'key_2' : 'abc',
    'value_1' : 5.5,
    'value_2' : 2.2
  },
  {
    'key_2' : 'xyz',
    'value_1' : 10.1,
    'value_2' : 13.5
  }
]

df1 = spark.createDataFrame(dataset_1)
print('df1')
df1.show()

df2 = spark.createDataFrame(dataset_2)
print('df2')
df2.show()

df1
+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
|  def|    100|    300|
+-----+-------+-------+

df2
+-----+-------+-------+
|key_2|value_1|value_2|
+-----+-------+-------+
|  abc|    5.5|    2.2|
|  xyz|   10.1|   13.5|
+-----+-------+-------+



**Left Semi join**

```Left Semi join``` - левое полусоединение, возвращает только те строки из левой таблицы, которые имеют соответствующее значение в правой таблице, при этом не возвращает значений из правой таблицы и может быть полезен для фильтрации данных

In [4]:
df1.join(df2, df1.key_1 == df2.key_2, 'left_semi').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
+-----+-------+-------+



In [5]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |abc|     10|     20|
# +---+-------+-------+

**Left Anti join**

```Left Anti join``` - левое антисоединение, возвращает только те строки из левой таблицы, которые не имеют соответствующих значений в правой таблице, при этом не возвращает значений из правой таблицы и может быть полезен для фильтрации данных

In [6]:
df1.join(df2, df1.key_1 == df2.key_2, 'left_anti').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  def|    100|    300|
+-----+-------+-------+



In [7]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |def|    100|    300|
# +---+-------+-------+

**Inner join**

```Inner join``` - внутреннее соединение, возвращает только строки, имеющие соответствие в обеих таблицах, а соединяет таблицы на основе общих значений в указанных столбцах

In [8]:
cond = [df1.value_1 != df2.value_2]
df1.join(df2, cond).show()

+-----+-------+-------+-----+-------+-------+
|key_1|value_1|value_2|key_2|value_1|value_2|
+-----+-------+-------+-----+-------+-------+
|  abc|     10|     20|  abc|    5.5|    2.2|
|  abc|     10|     20|  xyz|   10.1|   13.5|
|  def|    100|    300|  abc|    5.5|    2.2|
|  def|    100|    300|  xyz|   10.1|   13.5|
+-----+-------+-------+-----+-------+-------+



In [9]:
# Создайте Inner джойн с условиями ---hidden---, для df1 и df2, соответсвенно
# В  итоге получится таблица
# +---+-------+-------+---+-------+-------+
# |key|value_1|value_2|key|value_1|value_2|
# +---+-------+-------+---+-------+-------+
# |abc|     10|     20|abc|    5.5|    2.2|
# |abc|     10|     20|xyz|   10.1|   13.5|
# |def|    100|    300|abc|    5.5|    2.2|
# |def|    100|    300|xyz|   10.1|   13.5|
# +---+-------+-------+---+-------+-------+

### Виды JOIN в Spark:

[ INNER ] | CROSS | LEFT [ OUTER ] | [ LEFT ] SEMI | RIGHT [ OUTER ] | FULL [ OUTER ] | [ LEFT ] ANTI

[ссылка на документацию](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-join.html)

**```Inner join```** (внутреннее соединение) - это тип соединения, который возвращает только те строки, которые имеют соответствующие значения в обоих таблицах. Он соединяет две таблицы на основе общих значений в указанных столбцах.

**```Cross join```** (декартово произведение) - это тип соединения, который возвращает все возможные комбинации строк из двух таблиц. Он не требует указания столбцов для соединения и может быть очень ресурсоемким.

**```Left (outer) join```** (левое внешнее соединение) - это тип соединения, который возвращает все строки из левой таблицы и только те строки из правой таблицы, которые имеют соответствующие значения в указанных столбцах. Если значение не найдено в правой таблице, то оно будет заполнено NULL.

**```Left semi join```** (левое полу-соединение) - это тип соединения, который возвращает только те строки из левой таблицы, которые имеют соответствующие значения в правой таблице. Он не возвращает значения из правой таблицы и может быть полезен для фильтрации данных.

**```Right (outer) join```** (правое внешнее соединение) - это тип соединения, который возвращает все строки из правой таблицы и только те строки из левой таблицы, которые имеют соответствующие значения в указанных столбцах. Если значение не найдено в левой таблице, то оно будет заполнено NULL.

**```Full (outer) join```** (полное внешнее соединение) - это тип соединения, который возвращает все строки из обеих таблиц, а также значения, которые не имеют соответствия в другой таблице. При этом значения, которые не имеют соответствия в другой таблице, будут заполнены NULL.

**```Left anti join```** (левое анти-соединение) - это тип соединения, который возвращает только те строки из левой таблицы, которые не имеют соответствующих значений в правой таблице. Он не возвращает значения из правой таблицы и может быть полезен для фильтрации данных.